In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Generalized LASSO

Solve the total variation denoising problem based on Stephen Boyd, et al. ["Distributed optimization and statistical learning via the alternating direction method of multipliers".](https://dl.acm.org/doi/abs/10.1561/2200000016) Now Publishers Inc, 2011.


## Problem Description

$$\min \frac{1}{2} ||Ax-b||_2^2+\lambda||Fx||_1$$
Here we consider special case when $A$ is an indentity matrix and $F$ is the difference matrix, i.e.,
$$\min_x \frac{1}{2}||x-b||_2^2+\lambda\sum_{i=1}^{n-1}|x_{i+1}-x_i|$$

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path. 

In [2]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Data Initialization 
Specify torch device, and generate data

In [3]:
device = torch.device("cpu")
n = 80
eta = 0.5  # parameter for penalty term
torch.manual_seed(1)
b = torch.rand(n, 1)
pos_one = torch.ones(n - 1)
neg_one = -torch.ones(n - 1)
F = torch.zeros(n - 1, n)
F[:, 0 : n - 1] += torch.diag(neg_one, 0)
F[:, 1:n] += torch.diag(pos_one, 0)

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
F = F.to(
    device=device, dtype=torch.double
)  # double precision requireed in torch operations
b = b.to(device=device, dtype=torch.double)

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [4]:
# variables and corresponding dimensions.
var_in = {"x": [n, 1]}


def user_fn(X_struct, F, b):
    x = X_struct.x

    # objective function
    f = (x - b).T @ (x - b) + eta * torch.norm(F @ x, p=1)

    # inequality constraint, matrix form
    ci = None
    # equality constraint
    ce = None

    return [f, ci, ce]


comb_fn = lambda X_struct: user_fn(X_struct, F, b)

## User Options
Specify user-defined options for PyGRANSO

In [5]:
opts = pygransoStruct()
opts.torch_device = device
opts.x0 = torch.ones((n, 1)).to(device=device, dtype=torch.double)
opts.print_level = 1
opts.print_frequency = 10

## Main Algorithm

In [6]:
start = time.time()
cpu_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
══════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                          ║ 
Version 1.2.0                                                                                 ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang               ║ 
══════════════════════════════════════════════════════════════════════════════════════════════╣
Problem specifications:  

## CUDA Computations

In [7]:
device = torch.device("cuda")
n = 80
eta = 0.5  # parameter for penalty term
torch.manual_seed(1)
b = torch.rand(n, 1)
pos_one = torch.ones(n - 1)
neg_one = -torch.ones(n - 1)
F = torch.zeros(n - 1, n)
F[:, 0 : n - 1] += torch.diag(neg_one, 0)
F[:, 1:n] += torch.diag(pos_one, 0)

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
F = F.to(
    device=device, dtype=torch.double
)  # double precision requireed in torch operations
b = b.to(device=device, dtype=torch.double)

In [8]:
# variables and corresponding dimensions.
var_in = {"x": [n, 1]}


def user_fn(X_struct, F, b):
    x = X_struct.x

    # objective function
    f = (x - b).T @ (x - b) + eta * torch.norm(F @ x, p=1)

    # inequality constraint, matrix form
    ci = None
    # equality constraint
    ce = None

    return [f, ci, ce]


comb_fn = lambda X_struct: user_fn(X_struct, F, b)

In [9]:
opts = pygransoStruct()
opts.torch_device = device
opts.x0 = torch.ones((n, 1)).to(device=device, dtype=torch.double)
opts.print_level = 1
opts.print_frequency = 10

In [10]:
start = time.time()
cuda_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
══════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                          ║ 
Version 1.2.0                                                                                 ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang               ║ 
══════════════════════════════════════════════════════════════════════════════════════════════╣
Problem specifications:  

## Testing

In [11]:
torch.testing.assert_close(
    cpu_soln.final.x,
    cuda_soln.final.x.cpu(),
    atol=1e-6,
    rtol=1e-6,
)